In [ ]:
## Imports

# import python libraries  
import numpy as np
import pandas as pd
import sys
import os
import time
import pickle
import tensorflow as tf
import time
from importlib import reload
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from scripts.splitting import split
from scripts.data_wrangling import import_prot
from scripts.feature_selection import fs_landmark
from scripts.data_preparation import data_prep
from scripts.data_selection import data_indexing
from scripts.mean_model import MeanModel
from scripts.tf_DRP_model import train_model, train_model_cv, train_model_XGBr, prediction_metrics

tf.keras.utils.set_random_seed(42) # set random seeds for Python, NumPy, and TensorFlow

print('Imports: Done')


# Import and wrangle proteomics and GDSC1 drug data
prot_df,drug_df,drug_matrix,_all_cls,_all_drugs,common_ind = import_prot()
print('Data Wrangling: Done')

## Feature Selection 

# create list of landmark gene proteins
landmark_proteins = fs_landmark(prot_df)

# reduce remaining features down by feature selection
prot_df = prot_df.filter(landmark_proteins,axis=1)

## Data Preparation & Selection
x_drug, x_all, y_series = data_prep(drug_df,prot_df,common_ind)
drug_cl_pairs =  y_series.index

# Cancer blind train-test split
rand_seed = 42 # set random seed for train-test split
train_size = 0.75 # set train-test fraction

train_pairs, test_pairs = split(rand_seed, _all_cls, _all_drugs, drug_cl_pairs, 
                                train_size, split_type='cblind')

# index for each set of train-test pairs
xo_train, xd_train, y_train, xo_test, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all,x_drug,y_series) # select train and test

## XGBoost model

In [ ]:
# create model and set parameters
regressor = XGBRegressor(tree_method="hist", seed = 42, 
                        max_depth = 350, 
                        n_estimators = 630, 
                        eta = 0.04,  
                        min_child_weight = 3, 
                        colsample_bytree = 0.2, 
                        reg_alpha = 0.1) 

# run model and evaluate performance
train_model_XGBr(regressor, x_train = [xo_train, xd_train], 
               x_test = [xo_test, xd_test], 
               y_train = y_train, y_test = y_test)

## Neural Network 

In [ ]:
## Model Building

# Learning rate schedule function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


## CNN using functional API
def build_model(learning_rate=1e-2, momentum=0.9, seed=42):

    # set weight initialiser
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    
    # omics data input
    x_input = layers.Input(shape=(xo_train.shape[1],1))
    # 1st convolution layer
    x = layers.Conv1D(filters=8, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_input) 
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1,seed=seed)(x)
    # 2nd convolution layer
    x = layers.Conv1D(filters=16, kernel_size=4, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1,seed=seed)(x)
    x = layers.Flatten()(x)

    # FC layer for xo_train
    x = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5,seed=seed)(x)
    x = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5,seed=seed)(x)
    
    # one-hot encoded drug data + dense layer
    y_input = layers.Input(shape=(xd_train.shape[1]))
    y = Dense(64, kernel_initializer=initializer, activation="relu")(y_input) 
    
    # Concatenate omics and encoded drug data
    z = layers.concatenate([x, y])
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(1, kernel_initializer=initializer)(z)

    model = tf.keras.Model([x_input, y_input], z)
    
    opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, momentum=momentum)

    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

print('Model Building: Done')

In [ ]:
# train model without cross-validation

m_func = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler
epochs = 100

train_model(m_func,lr_scheduler, 
          x_train = [xo_train, xd_train], x_test = [xo_test, xd_test], 
          y_train = y_train, y_test = y_test, 
          epochs = epochs)

In [ ]:
# train model with cross-validation

epochs = 100
rand_seed_list = [913,425]
model = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler

train_model_cv(model,lr_scheduler,
             train_pairs,x_all,x_drug,y_series,
             epochs,rand_seed_list)